In [1]:
# env : pixlake
# we focuing on pyspark dataframe processing
# documentation https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame
%load_ext autoreload
%autoreload 2

In [2]:
# make you auto compeletion faster
# https://stackoverflow.com/questions/40536560/ipython-and-jupyter-autocomplete-not-working
%config Completer.use_jedi = False

In [3]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['SPARK_HOME'] = '/opt/spark/versions/spark-2.3'

In [4]:
from pyspark.sql import SparkSession as Session
from pyspark import SparkConf as Conf
from pyspark.sql import functions as F, Window as W
C = F.col

In [5]:
conf = (Conf()
    .set('spark.sql.sources.partitionOverwriteMode', 'dynamic')
    .set('spark.driver.memory', '5g')
    .set('spark.driver.maxResultSize', '2g')
   )

In [6]:
spark = (Session
     .builder
     .appName('pyspark-challenge')
     .master('local[2]')
     .config(conf=conf)
     .getOrCreate())

# Creating DataFrame

In [7]:
# 0. knowing what spark session can do and its version
print(dir(spark), f'your spark version : {spark.version}'
      , sep='\n\n')

['Builder', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_convert_from_pandas', '_createFromLocal', '_createFromRDD', '_create_from_pandas_with_arrow', '_get_numpy_record_dtype', '_inferSchema', '_inferSchemaFromList', '_instantiatedSession', '_jsc', '_jsparkSession', '_jvm', '_jwrapped', '_repr_html_', '_sc', '_wrapped', 'builder', 'catalog', 'conf', 'createDataFrame', 'newSession', 'range', 'read', 'readStream', 'sparkContext', 'sql', 'stop', 'streams', 'table', 'udf', 'version']

your spark version : 2.3.4


In [8]:
# 1. read data from csv
print(os.listdir('../data'))
df_from_csv_1 = spark.read.csv('../data/zipcodes.csv',
                               header=True,
                              inferSchema=True)
df_from_csv_1.printSchema()
df_from_csv_1.limit(5).toPandas()

['zipcode1.json', 'titanic_train.csv', 'small_zipcode.csv', 'Meteorite_Landings.csv', 'zipcodes.csv', 'zipcodes.json', 'webpage_1.txt', 'multiline-zipcode.json', 'simple_text.txt', 'zipcode2.json', 'titanic_test.csv']
root
 |-- RecordNumber: integer (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- ZipCodeType: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- LocationType: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Xaxis: double (nullable = true)
 |-- Yaxis: double (nullable = true)
 |-- Zaxis: double (nullable = true)
 |-- WorldRegion: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- LocationText: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Decommisioned: boolean (nullable = true)
 |-- TaxReturnsFiled: integer (nullable = true)
 |-- EstimatedPopulation: integer (nullable = true)
 |-- TotalWages: integer (nullable = t

,RecordNumber,Zipcode,ZipCodeType,City,State,LocationType,Lat,Long,Xaxis,Yaxis,Zaxis,WorldRegion,Country,LocationText,Location,Decommisioned,TaxReturnsFiled,EstimatedPopulation,TotalWages,Notes
0,1,704,STANDARD,PARC PARQUE,PR,NOT ACCEPTABLE,17.96,-66.22,0.38,-0.87,0.30,NA,US,"Parc Parque, PR",NA-US-PR-PARC PARQUE,False,NaN,NaN,NaN,None
1,2,704,STANDARD,PASEO COSTA DEL SUR,PR,NOT ACCEPTABLE,17.96,-66.22,0.38,-0.87,0.30,NA,US,"Paseo Costa Del Sur, PR",NA-US-PR-PASEO COSTA DEL SUR,False,NaN,NaN,NaN,None
2,10,709,STANDARD,BDA SAN LUIS,PR,NOT ACCEPTABLE,18.14,-66.26,0.38,-0.86,0.31,NA,US,"Bda San Luis, PR",NA-US-PR-BDA SAN LUIS,False,NaN,NaN,NaN,None
3,61391,76166,UNIQUE,CINGULAR WIRELESS,TX,NOT ACCEPTABLE,32.72,-97.31,-0.10,-0.83,0.54,NA,US,"Cingular Wireless, TX",NA-US-TX-CINGULAR WIRELESS,False,NaN,NaN,NaN,None
4,61392,76177,STANDARD,FORT WORTH,TX,PRIMARY,32.75,-97.33,-0.10,-0.83,0.54,NA,US,"Fort Worth, TX",NA-US-TX-FORT WORTH,False,2126.0,4053.0,122396986.0,None


In [9]:
# 2 read data from json
print(os.listdir('../data'))
print(dir(spark.read))
# 沒有infer_schema
df_from_json = spark.read.json('../data/zipcodes.json')
df_from_json.printSchema()
df_from_json.limit(5).toPandas()

['zipcode1.json', 'titanic_train.csv', 'small_zipcode.csv', 'Meteorite_Landings.csv', 'zipcodes.csv', 'zipcodes.json', 'webpage_1.txt', 'multiline-zipcode.json', 'simple_text.txt', 'zipcode2.json', 'titanic_test.csv']
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_df', '_jreader', '_set_opts', '_spark', 'csv', 'format', 'jdbc', 'json', 'load', 'option', 'options', 'orc', 'parquet', 'schema', 'table', 'text']
root
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Decommisioned: boolean (nullable = true)
 |-- EstimatedPopulation: long (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- LocationText: string (nullable = 

,City,Country,Decommisioned,EstimatedPopulation,Lat,Location,LocationText,LocationType,Long,Notes,RecordNumber,State,TaxReturnsFiled,TotalWages,WorldRegion,Xaxis,Yaxis,Zaxis,ZipCodeType,Zipcode
0,PARC PARQUE,US,False,NaN,17.96,NA-US-PR-PARC PARQUE,"Parc Parque, PR",NOT ACCEPTABLE,-66.22,None,1,PR,NaN,NaN,NA,0.38,-0.87,0.30,STANDARD,704
1,PASEO COSTA DEL SUR,US,False,NaN,17.96,NA-US-PR-PASEO COSTA DEL SUR,"Paseo Costa Del Sur, PR",NOT ACCEPTABLE,-66.22,None,2,PR,NaN,NaN,NA,0.38,-0.87,0.30,STANDARD,704
2,BDA SAN LUIS,US,False,NaN,18.14,NA-US-PR-BDA SAN LUIS,"Bda San Luis, PR",NOT ACCEPTABLE,-66.26,None,10,PR,NaN,NaN,NA,0.38,-0.86,0.31,STANDARD,709
3,CINGULAR WIRELESS,US,False,NaN,32.72,NA-US-TX-CINGULAR WIRELESS,"Cingular Wireless, TX",NOT ACCEPTABLE,-97.31,None,61391,TX,NaN,NaN,NA,-0.10,-0.83,0.54,UNIQUE,76166
4,FORT WORTH,US,False,4053.0,32.75,NA-US-TX-FORT WORTH,"Fort Worth, TX",PRIMARY,-97.33,None,61392,TX,2126.0,122396986.0,NA,-0.10,-0.83,0.54,STANDARD,76177


In [10]:
# 3 create dataframe from rdd list
columns = ["language","user_counts"]
data = [
    ("Java","20000"),
    ("Python","100000"),
    ("Scala","3000")
       ]
# 先分散到rdd
rdd = spark.sparkContext.parallelize(data)
print(dir(rdd), type(rdd), sep='\n\n')
print()
df_from_rdd = rdd.toDF(schema=columns)
df_from_rdd.show(n=5)

# 直接create，讓spark dataframe進行分散
df = spark.createDataFrame(data=data,schema=columns)
df.show(n=5)



['__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_computeFractionForSampleSize', '_defaultReducePartitions', '_id', '_jrdd', '_jrdd_deserializer', '_memory_limit', '_pickled', '_reserialize', '_to_java_object_rdd', 'aggregate', 'aggregateByKey', 'cache', 'cartesian', 'checkpoint', 'coalesce', 'cogroup', 'collect', 'collectAsMap', 'combineByKey', 'context', 'count', 'countApprox', 'countApproxDistinct', 'countByKey', 'countByValue', 'ctx', 'distinct', 'filter', 'first', 'flatMap', 'flatMapValues', 'fold', 'foldByKey', 'foreach', 'foreachPartition', 'fullOuterJoin', 'getCheckpointFile', 'getNumPartitions', 'getStorageLevel', 'glom', 'groupBy', 'groupByKey', 'groupWith

In [11]:
# 4 create 5 row fake data using spark range
print(dir(spark))
print(type(spark.range(start=0,end=10)))
columns = ['row_number']
single_column_df = spark.range(start=0,end=10)
single_column_df.show(n=5)

['Builder', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_conf', '_convert_from_pandas', '_createFromLocal', '_createFromRDD', '_create_from_pandas_with_arrow', '_get_numpy_record_dtype', '_inferSchema', '_inferSchemaFromList', '_instantiatedSession', '_jsc', '_jsparkSession', '_jvm', '_jwrapped', '_repr_html_', '_sc', '_wrapped', 'builder', 'catalog', 'conf', 'createDataFrame', 'newSession', 'range', 'read', 'readStream', 'sparkContext', 'sql', 'stop', 'streams', 'table', 'udf', 'version']
<class 'pyspark.sql.dataframe.DataFrame'>
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+
only showing top 5 rows



In [12]:
# 6 create empty dataframe

columns = ["language","user_counts"]

# empty RDD + schema won't work
# df_1 = spark.createDataFrame(data=spark.sparkContext.emptyRDD(),
#                              schema=columns)
# df_1.show(n=5)

# empty list -> rdd -> df won't work
# df2 = spark.sparkContext.parallelize([]).toDF(columns)

# df3 = spark.createDataFrame([]) # won't work this version

In [13]:
# 7 get dataframe shape
print(df.count(), len(df.columns))

3 2


In [14]:
# 8 knowing what methods are supported by sql.function
print(dir(F))

['AutoBatchedSerializer', 'Column', 'DataFrame', 'DataType', 'PandasUDFType', 'PickleSerializer', 'PythonEvalType', 'SparkContext', 'StringType', 'UserDefinedFunction', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_binary_mathfunctions', '_collect_list_doc', '_collect_set_doc', '_create_binary_mathfunction', '_create_function', '_create_udf', '_create_window_function', '_functions', '_functions_1_4', '_functions_1_6', '_functions_2_1', '_functions_deprecated', '_lit_doc', '_message', '_string_functions', '_test', '_to_java_column', '_to_seq', '_window_functions', '_wrap_deprecated_function', 'abs', 'acos', 'add_months', 'approxCountDistinct', 'approx_count_distinct', 'array', 'array_contains', 'asc', 'ascii', 'asin', 'atan', 'atan2', 'avg', 'base64', 'bin', 'bitwiseNOT', 'blacklist', 'broadcast', 'bround', 'cbrt', 'ceil', 'coalesce', 'col', 'collect_list', 'collect_set', 'column', 'concat', 'concat_ws', 'conv', 'c

In [15]:
# 9 add const column to a existing dataframe

columns = ["language","user_counts"]
data = [
    ("Java","20000"),
    ("Python","100000"),
    ("Scala","3000")
       ]
df = spark.createDataFrame(data=data,schema=columns)
df = (
    df.withColumn("new_column",F.lit("ABC")) 
    # F.lit means literal, retrurn a column
)
df.show(n=5)

+--------+-----------+----------+
|language|user_counts|new_column|
+--------+-----------+----------+
|    Java|      20000|       ABC|
|  Python|     100000|       ABC|
|   Scala|       3000|       ABC|
+--------+-----------+----------+



In [16]:
# 10 add a row_id column from a exisiting dataframe
# https://stackoverflow.com/questions/53082891/adding-a-unique-consecutive-row-number-to-dataframe-in-pyspark
columns = ["language","user_counts"]
data = [
    ("Java","20000"),
    ("Python","100000"),
    ("Scala","3000")
       ]
df_1 = spark.createDataFrame(data=data,schema=columns)
df_1 = (
    df_1.withColumn("index", 
                  F.row_number().over(
                      W.orderBy(F.monotonically_increasing_id() - 1)
                  )
                 )
    # F.monotonically_increasing_id does not give 1 ~ N
    # So we use window function to work around
)
df_1.show(n=5)

# Mre clear way to do that
df_2 = spark.createDataFrame(data=data,schema=columns)
w = W.orderBy(F.lit('A'))
df_2 = (
    df_2.withColumn("row_num", F.row_number().over(w))
)
df_2.show(n=5)

+--------+-----------+-----+
|language|user_counts|index|
+--------+-----------+-----+
|    Java|      20000|    1|
|  Python|     100000|    2|
|   Scala|       3000|    3|
+--------+-----------+-----+

+--------+-----------+-------+
|language|user_counts|row_num|
+--------+-----------+-------+
|    Java|      20000|      1|
|  Python|     100000|      2|
|   Scala|       3000|      3|
+--------+-----------+-------+



In [17]:
# 10 add a random number to a exisit column
df = (
#     df.withColumn('random_number', F.when(F.rand() > 0.5, 1).otherwise(0))
        df.withColumn('random_number', F.rand())
)

df.show(n=5)

+--------+-----------+----------+------------------+
|language|user_counts|new_column|     random_number|
+--------+-----------+----------+------------------+
|    Java|      20000|       ABC|0.7336041953493867|
|  Python|     100000|       ABC|0.7321022661102865|
|   Scala|       3000|       ABC|0.6141542697675181|
+--------+-----------+----------+------------------+



In [18]:
# 11 add a binary 0, 1 based on condition to an exisit column
df = (
    df.withColumn('binary_cut_05',F.when(F.rand() > 0.5, 1).otherwise(0))
)
df.limit(5).toPandas()

,language,user_counts,new_column,random_number,binary_cut_05
0,Java,20000,ABC,0.733604,1
1,Python,100000,ABC,0.732102,0
2,Scala,3000,ABC,0.614154,1


In [19]:
# 12 create a dataframe contains row_index and fake data

columns = ["row_id","language","user_counts"]
data = [
    (0, "Java","20000"),
    (1, "Python","100000"),
    (2, "Scala","3000")
       ]

df = spark.createDataFrame(data=data, schema=columns)
df.show(n=5)


+------+--------+-----------+
|row_id|language|user_counts|
+------+--------+-----------+
|     0|    Java|      20000|
|     1|  Python|     100000|
|     2|   Scala|       3000|
+------+--------+-----------+



In [34]:
# 13 construct a complex data for spark dataframe
# using StructType

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Case 1
data = [
    ("James","","Smith","36636","M", 3000),
    ("Michael","Rose","","40288","M", 4000),
    ("Robert","","Williams","42114","M", 4000),
    ("Maria","Anne","Jones","39192","F", 4000),
    ("Jen","Mary","Brown","","F", -1)
]

schema = StructType([
    StructField("firstname",StringType(), True), # Nullable True
    StructField("middlename",StringType(), True),
    StructField("lastname",StringType(), True),
    StructField("id",StringType(), True),
    StructField("gender",StringType(), True),
    StructField("salary", IntegerType(), True)
    ])
df = spark.createDataFrame(data=data,schema=schema)
df.show(n=5)

# Case 2

struct_data = [
    (("James","","Smith"),"36636","M", 3000),
    (("Michael","Rose",""),"40288","M", 4000),
    (("Robert","","Williams"),"42114","M", 4000),
    (("Maria","Anne","Jones"),"39192","F", 4000),
    (("Jen","Mary","Brown"),"","F", -1)
]

structure_schema = StructType([
    StructField('name',
        StructType([
            StructField("firstname",StringType(), True), # Nullable True
            StructField("middlename",StringType(), True),
            StructField("lastname",StringType(), True),
    ])),
    StructField("id",StringType(), True),
    StructField("gender",StringType(), True),
    StructField("salary", IntegerType(), True)
    ])

df = spark.createDataFrame(data=struct_data, schema=structure_schema)
df.show(n=5)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|   id|gender|salary|
+---------+----------+--------+-----+------+------+
|    James|          |   Smith|36636|     M|  3000|
|  Michael|      Rose|        |40288|     M|  4000|
|   Robert|          |Williams|42114|     M|  4000|
|    Maria|      Anne|   Jones|39192|     F|  4000|
|      Jen|      Mary|   Brown|     |     F|    -1|
+---------+----------+--------+-----+------+------+

+--------------------+-----+------+------+
|                name|   id|gender|salary|
+--------------------+-----+------+------+
|    [James, , Smith]|36636|     M|  3000|
|   [Michael, Rose, ]|40288|     M|  4000|
|[Robert, , Williams]|42114|     M|  4000|
|[Maria, Anne, Jones]|39192|     F|  4000|
|  [Jen, Mary, Brown]|     |     F|    -1|
+--------------------+-----+------+------+



In [21]:
# 14 construct a complex data for spark dataframe
# using ArratyType

In [ ]:
# 15 construct a complex data for spark dataframe
# using MapType

In [22]:
# 16 create a datetime column for spark dataframe

# Column Operations

In [23]:
# 1 create new column based on original column

columns = ["language", "user_counts"]
data = [
    ("Java",20000),
    ("Python",100000),
    ("Scala",3000)
]

df = spark.createDataFrame(data=data, schema=columns)
df = (
    df.withColumn("user_count_100", C("user_counts") * 100)\
    .withColumn("user_count_log", F.log10(C("user_counts")))
)

df.show(n=5)

+--------+-----------+--------------+------------------+
|language|user_counts|user_count_100|    user_count_log|
+--------+-----------+--------------+------------------+
|    Java|      20000|       2000000| 4.301029995663981|
|  Python|     100000|      10000000|               5.0|
|   Scala|       3000|        300000|3.4771212547196626|
+--------+-----------+--------------+------------------+



In [40]:
# 2 rename, drop, add constant column to existing dataframe
# https://stackoverflow.com/questions/34077353/how-to-change-dataframe-column-names-in-pyspark
columns = ["language", "user_counts"]
data = [
    ("Java",20000),
    ("Python",100000),
    ("Scala",3000)
]

df = spark.createDataFrame(data, columns)
df = (
    df.withColumn('const_col',F.lit('ABC'))\
    .withColumnRenamed("language","lang")\
    .drop("user_counts")
)
df.show(n=5)

+------+---------+
|  lang|const_col|
+------+---------+
|  Java|      ABC|
|Python|      ABC|
| Scala|      ABC|
+------+---------+



In [56]:
# 3 creating a new dynamic column(if else condition based on old column)
# Case 1
columns = ["language", "user_counts"]
data = [
    ("Java",20000),
    ("Python",100000),
    ("Scala",3000)
]

df = spark.createDataFrame(data, columns)

df = (
    df.withColumn("is_many_users",
                  F.when(C('user_counts') > 50000, 1).otherwise(0)
                 )
)

df.show(n=5)

# Case 2

data = [
    ("James","","Smith","36636","M", 3000),
    ("Michael","Rose","","40288","M", 4000),
    ("Robert","","Williams","42114","M", 4000),
    ("Maria","Anne","Jones","39192","F", 4000),
    ("Jen","Mary","Brown","","F", -1)
]

schema = StructType([
    StructField("firstname",StringType(), True), # Nullable True
    StructField("middlename",StringType(), True),
    StructField("lastname",StringType(), True),
    StructField("id",StringType(), True),
    StructField("gender",StringType(), True),
    StructField("salary", IntegerType(), True)
    ])
df = spark.createDataFrame(data=data,schema=schema)

is_full_name_exist = (C("firstname").isNull() & C("middlename").isNull() & C("lastname").isNull())
# a bug here, try replace empty string to None
df = (
    df.withColumn("full_name",
                  F.when(
                      is_full_name_exist,
                      F.concat(C("firstname"), F.lit(' '),
                               C("middlename"), F.lit(' '),
                               C("lastname"))
                  ).otherwise(F.lit('N/A'))
                 )
)
df.show(n=5)



+--------+-----------+-------------+
|language|user_counts|is_many_users|
+--------+-----------+-------------+
|    Java|      20000|            0|
|  Python|     100000|            1|
|   Scala|       3000|            0|
+--------+-----------+-------------+

+---------+----------+--------+-----+------+------+---------+
|firstname|middlename|lastname|   id|gender|salary|full_name|
+---------+----------+--------+-----+------+------+---------+
|    James|          |   Smith|36636|     M|  3000|      N/A|
|  Michael|      Rose|        |40288|     M|  4000|      N/A|
|   Robert|          |Williams|42114|     M|  4000|      N/A|
|    Maria|      Anne|   Jones|39192|     F|  4000|      N/A|
|      Jen|      Mary|   Brown|     |     F|    -1|      N/A|
+---------+----------+--------+-----+------+------+---------+



In [64]:
# 4 select columns which also exist on another dataframe

columns_1 = ["language", "user_counts"]
data_1 = [
    ("Java",20000),
    ("Python",100000),
    ("Scala",3000)
]

columns_2 = ["language", "user_counts","note"]
data_2 = [
    ("Java",20000,"nothing"),
    ("Python",100000,"nothing"),
    ("Scala",3000,"nothing")
]

df_1 = spark.createDataFrame(data_1, columns_1)
df_2 = spark.createDataFrame(data_2, columns_2)

# union columns
same_cols = [F.col(c) for c in df_2.columns if c in df_1.columns]
print(same_cols, type(same_cols), type(same_cols[0]))

df_same_col = df_1.select(*same_cols)
df_same_col.show(n=5)

[Column<b'language'>, Column<b'user_counts'>] <class 'list'> <class 'pyspark.sql.column.Column'>
+--------+-----------+
|language|user_counts|
+--------+-----------+
|    Java|      20000|
|  Python|     100000|
|   Scala|       3000|
+--------+-----------+



In [25]:
# 5 string concat two column values to a new column from an existing dataframe

In [26]:
# 6 cut of left 3 char of specific column to a new column from an existing dataframe

In [27]:
# 7 convert string type column to int/float type column

In [28]:
# 8 convert string type column to datetime type column